# Género y tecnología: ¿qué eligen los estudiantes?

In [22]:
from collections import namedtuple

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from scipy import stats
import seaborn as sns
import matplotlib
import ipywidgets as widgets
from ipywidgets import interact
import folium
from folium.plugins import DualMap 
from datetime import datetime
from IPython.display import HTML, display

# módulos propios
from scripts import mapas, pygal_helpers, georef, download
from scripts.pygal_helpers import df_to_pygal_line, get_pygal_html
from scripts.helpers import read_csv, format_pct, get_bins

matplotlib.style.use("ggplot")

## Evolución de género en grupo de carreras

In [23]:
df_evolucion_carreras = read_csv('grupos_evolucion_genero.csv')
df_evolucion_grupos_carreras = read_csv('grupos_carreras_evolucion_genero.csv')

df_evolucion_carreras.head(10)

,anio,carrera_grupo,estudiantes_total,estudiantes_varones,estudiantes_mujeres,nuevos_inscriptos_total,nuevos_inscriptos_varones,nuevos_inscriptos_mujeres,egresados_total,egresados_varones,egresados_mujeres,estudiantes_mujeres_pct,inscripciones_mujeres_pct,estudiantes_hombres_pct
0,2010,Arquitectura,70587.0,26843.0,25641.0,13655.0,5266.0,5240.0,3166.0,1121.0,1309.0,0.363254,0.383742,0.380282
1,2010,Arte,84461.0,24721.0,35384.0,23097.0,7455.0,10122.0,2854.0,745.0,1321.0,0.418939,0.438239,0.292691
2,2010,Deporte,2621.0,1557.0,1064.0,794.0,471.0,323.0,162.0,90.0,72.0,0.405952,0.406801,0.594048
3,2010,Derecho,237773.0,87646.0,114635.0,56072.0,23004.0,27457.0,15903.0,5184.0,6912.0,0.482120,0.489674,0.368612
4,2010,Economicas,401354.0,158885.0,184338.0,95385.0,41997.0,43894.0,25750.0,10380.0,11933.0,0.459290,0.460177,0.395872
5,2010,Educacion,142273.0,35322.0,101339.0,43097.0,11735.0,29675.0,8569.0,1909.0,5994.0,0.712286,0.688563,0.248269
6,2010,Exactas,43302.0,17651.0,20001.0,10271.0,4196.0,4876.0,2523.0,1008.0,1331.0,0.461896,0.474735,0.407626
7,2010,Idiomas,13885.0,1909.0,9710.0,4055.0,665.0,2739.0,754.0,71.0,617.0,0.699316,0.675462,0.137486
8,2010,Ingeniería,104596.0,81481.0,13967.0,22107.0,16686.0,3252.0,4556.0,3488.0,650.0,0.133533,0.147103,0.779007
9,2010,Letras,340.0,71.0,269.0,76.0,21.0,55.0,17.0,1.0,16.0,0.791176,0.723684,0.208824


In [24]:
# NOTAS: Comparacion de valores absolutos de hombres y mujeres - nuevas inscripciones/ filtro por carrera (eje primaro y secundario??)

In [25]:
df_evolucion_carreras = df_evolucion_carreras[df_evolucion_carreras.anio != 2010]

In [26]:
df_evolucion_grupos_carreras = df_evolucion_grupos_carreras[df_evolucion_grupos_carreras.anio !=2010]

### ¿Cómo evolucionó la proporción de estudiantes femeninos en distintas disciplinas?

In [27]:
# gráfico de evolución hombres / mujeres por grupo de carreras

# widget: seleccionar grupo temático de carreras
variables_carreras_evolucion = tuple(df_evolucion_carreras.carrera_grupo.unique())
widget_grupos=widgets.SelectMultiple(
    options=sorted(variables_carreras_evolucion),
    value=["Tecnologia"],
    description='Areas Tematicas',
    disabled=False
)

# widget: elegir indicador
variables_indicadores_line = {
    "Ratio mujeres": "estudiantes_mujeres_pct",
    "Ratio hombres": "estudiantes_hombres_pct"
}
widget_indicador = widgets.ToggleButtons(
    options=variables_indicadores_line.items(),
    description="Indicador : ",
    value="estudiantes_mujeres_pct"
)

@interact(carrera_grupo=widget_grupos, indicador=widget_indicador)
def generate_chart_evolucion_grupos(carrera_grupo, indicador):
    
    serie_grupo_carrera = df_evolucion_carreras[
        df_evolucion_carreras.carrera_grupo.isin(carrera_grupo)][[
            "anio", 'carrera_grupo',  indicador]]

    serie_grupo_carrera[indicador]=serie_grupo_carrera[indicador].apply(format_pct)
    
    df_pivot_serie_grupo_carrera = serie_grupo_carrera.pivot_table(
        index='anio', columns='carrera_grupo', values=indicador)
    
    return HTML(get_pygal_html(df_to_pygal_line(df_pivot_serie_grupo_carrera)))
        
#NOTAS: aclarar en nota que los valores son porcentajes

interactive(children=(SelectMultiple(description='Areas Tematicas', index=(17,), options=('Arquitectura', 'Art…

In [28]:
# gráfico de evolución de las carreras dentro de un grupo

@interact(carrera_grupo=widget_grupos) #widget diferente
def generate_chart_evolucion_grupos(carrera_grupo):
    
    serie_grupo_carrera = df_evolucion_grupos_carreras[
        df_evolucion_grupos_carreras.carrera_grupo.isin(carrera_grupo)][[
        "anio", 'carrera_nombre',  "estudiantes_mujeres_pct"]]
    df_pivot_serie_grupo_carrera = serie_grupo_carrera.pivot_table(
        index='anio', columns='carrera_nombre', values='estudiantes_mujeres_pct')
    
    return HTML(get_pygal_html(df_to_pygal_line(df_pivot_serie_grupo_carrera)))
        

interactive(children=(SelectMultiple(description='Areas Tematicas', index=(17,), options=('Arquitectura', 'Art…

In [29]:
#agregar mas de un filtro para ir comparando
#modificar en ETL abogacia y derecho
#apertura por carreras para entender donde hace el pico y decrecimiento tech - cambiar pKEY por carrera y grupo carrera - grafico exploratorio
#segunda tabla de ETL donde haya grupo carrera (carrera) + universidades
#grafico con valores absolutos - apilados o algo que represente orden de magnitud año por default 

## Comparación de género por provincias y universidades

In [30]:
df_universidades = pd.read_csv('data/output/universidades_mapa.csv')
df_evolucion_ubicacion = pd.read_csv('data/output/ubicacion_evolucion_genero.csv', 
                                     dtype={'provincia_id': str})
df_evolucion_ubicacion['provincia_id'] = df_evolucion_ubicacion.provincia_id.str.zfill(2)

In [31]:
argentina_coords = [-40, -64]
osm_no_labels = "https://tiles.wmflabs.org/osm-no-labels/{z}/{x}/{y}.png"
ign_labels = "http://wms.ign.gob.ar/geoserver/gwc/service/tms/1.0.0/capabaseargenmap@EPSG%3A3857@png/{z}/{x}/{y}.png"

## Indicadores por provincia

In [32]:
INDICADORES_HEATMAP = {
    "estudiantes_mujeres_pct" : "Ratio de estudiantes mujeres",
    "estudiantes_grupo_pct" : "Ratio de estudiantes",
    "estudiantes_varones" : "Total de estudiantes hombres",
    #"estudiantes_mujeres" : "Total de estudiantes mujeres", 
    #"inscriptos_mujeres_pct" : "Ratio de inscriptas mujeres ",
    #"inscriptos_hombres_pct" : "Ratio de inscriptas hombres ",
    #"estudiantes_hombres_pct" : "Ratio de estudiantes hombres ",
}
variables_indicadores = {
    "Ratio mujeres": "estudiantes_mujeres_pct",
    "Ratio grupo": "estudiantes_grupo_pct",
    "Est. hombres": "estudiantes_varones"
}

variables_carreras_heatmap = sorted(df_evolucion_ubicacion.carrera_grupo.unique())
variables_anio_heatmap = sorted(df_evolucion_ubicacion.anio.unique())

widget_indicador = widgets.ToggleButtons(
    options=variables_indicadores.items(),
    description="Indicador : ",
    value="estudiantes_mujeres_pct"
)

widget_carreras = widgets.Dropdown(
    options=variables_carreras_heatmap,
    description="Grupo : ",
    value="Tecnologia"
)

widget_anio = widgets.Dropdown(
    options=variables_anio_heatmap,
    description="Año : ",
    value=2016
)

widget_scale = widgets.RadioButtons(
    options=['Cuantiles', 'Fija', 'Relativa'],
    description='Escala :',
    value='Relativa'
)

@interact(anio=widget_anio,
          carrera_grupo=widget_carreras,
          indicador=widget_indicador,
          scale_type=widget_scale
         )
def generate_heat_map(anio, carrera_grupo, indicador, scale_type=None):
    mapa_heatmap = folium.Map(location=argentina_coords, zoom_start=4)
    folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa_heatmap)
    
    df_filter = df_evolucion_ubicacion[(df_evolucion_ubicacion.anio == anio)&
                           (df_evolucion_ubicacion.carrera_grupo == carrera_grupo)]
    
    if scale_type == 'Cuantiles':
        bins = list(df_filter[indicador].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
    elif scale_type == 'Fija':
        bins = get_bins(df_evolucion_ubicacion[indicador].min(), df_evolucion_ubicacion[indicador].max())
    else:
        bins = 5
       
    indicador_desc = INDICADORES_HEATMAP[indicador]
    folium.Choropleth(
        geo_data="data/input/provincias.geojson", # la capa de provincias en GeoJson
        data=df_filter, # DataFrame con los datos a graficar
        columns=['provincia_id', indicador], # columna con id de geometrías, columna con el dato
        key_on='feature.properties.id', # campo del geojson que tiene el id de las geometrías
        fill_color='YlGn', # escala de colores a usar
        #fill_opacity=0.7, # opacidad del color de relleno
        #line_opacity=0.2, # opacidad de las líneas que separan los polígonos
        legend_name='{} en {} (Año {})'.format(indicador_desc, carrera_grupo, anio), # título de la leyenda de la escala
        bins=bins
    ).add_to(mapa_heatmap)
    #mapas.add_labels_from_column(mapa_heatmap,df_evolucion_ubicacion,"nombre")

    return mapa_heatmap

# NOTAS
#Agregar un tooltip que muestre informacion de la provincia al pasar el cursor. Nombre provincia, estudiantes totales, año, porcentaje de mujeres, cantidad de universidades por provincia.
#Hacer zoom sobre la cuidad de buenos aires. Hacer un grafico nuevo
#min, max y media de la variable analizada por grupo.
#Agregar grafico de evolucion y/y entre provincias por grupo de carreras. Grafico de barras con la composicion de los grupos
#Cambiar los nombres a los indices porque son muy tecnicos. Podriamos usar indice de feminidad, grupo??, masculidad
#Se puede cambiar la escala cuando no hay datos?? (blancos o transparente)
#Tabla con visualizacon y que tenga un orden de ranking para poder identificar facilmente cual es la provincia que mas avanzo

interactive(children=(Dropdown(description='Año : ', index=6, options=(2010, 2011, 2012, 2013, 2014, 2015, 201…

## Indicadores por universidad

In [33]:
def mapear_universidades(mapa, df, lat_col, lon_col, institucion_nombre_col, carreras_grupos_col):
    """ cargar informacion al marker de forma masiva.

        Args:
            lat (float): latitud
            lon (float): longuitud
            institucion_nombre (str): nombre de la universidad
            carreras_grupos (list): lista con las areas de estudios ofrecidas

        """
    
    for row in df_universidades.iterrows():
        mapas.crear_marker(
            mapa,
            row[1]["lat"],
            row[1]["lon"],
            row[1]["institucion_nombre"],
            str(row[1]["carrera_grupo"]).split(",")
        )
        
#NOTAS: agregar feminidad por grupo carrera, elegir si queremos hacer selector por año o elegir 2016.    

In [34]:
mapa_markers = folium.Map(location=argentina_coords, zoom_start=4)
folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa_markers)

In [35]:
mapear_universidades(
    mapa_markers, df_universidades, 
    "lat","lon","institucion_nombre","carrera_grupo"
)

In [36]:
mapa_markers

# NOTAS
#Mapa + tabla "donde estudio" puedo hacer un filtro por grupo carrera (inverso a lo que estamos haciendo) 
#Cantidad de alumnas mujeres o comparacion de universidades
#agregar al tooltip el indice de feminidad

## Indicadores por universidad Y heatmap

In [37]:
def generate_heat_marker_map(anio, carrera_grupo, indicador, scale_type=None):
    mapa = folium.Map(location=argentina_coords, zoom_start=4)
    folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa)
    
    df_filter = df_evolucion_ubicacion[(df_evolucion_ubicacion.anio == anio)&
                           (df_evolucion_ubicacion.carrera_grupo == carrera_grupo)]
    
    if scale_type == 'Cuantiles':
        bins = list(df_filter[indicador].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
    elif scale_type == 'Fija':
        bins = get_bins(df_evolucion_ubicacion[indicador].min(), df_evolucion_ubicacion[indicador].max())
    else:
        bins = 5
       
    indicador_desc = INDICADORES_HEATMAP[indicador]
    folium.Choropleth(
        geo_data="data/input/provincias.geojson", # la capa de provincias en GeoJson
        data=df_filter, # DataFrame con los datos a graficar
        columns=['provincia_id', indicador], # columna con id de geometrías, columna con el dato
        key_on='feature.properties.id', # campo del geojson que tiene el id de las geometrías
        fill_color='YlGn', # escala de colores a usar
        #fill_opacity=0.7, # opacidad del color de relleno
        #line_opacity=0.2, # opacidad de las líneas que separan los polígonos
        legend_name='{} en {} (Año {})'.format(indicador_desc, carrera_grupo, anio), # título de la leyenda de la escala
        bins=bins
    ).add_to(mapa)
    #mapas.add_labels_from_column(mapa_heatmap,df_evolucion_ubicacion,"nombre")
    
    mapear_universidades(
    mapa, df_universidades, 
    "lat","lon","institucion_nombre","carrera_grupo")

    return mapa

In [38]:
mapa

NameError: name 'mapa' is not defined